In [ ]:
import  torch 
import  torchvision



In [2]:
x = torch.empty(5, 3)
print(x)

tensor([[7.5306e-39, 8.4490e-39, 1.0102e-38],
        [9.0919e-39, 1.0102e-38, 8.9082e-39],
        [9.2755e-39, 1.0837e-38, 8.4490e-39],
        [1.1112e-38, 1.0194e-38, 9.0919e-39],
        [8.4490e-39, 6.9796e-39, 1.0561e-38]])


In [3]:
x = torch.rand(5, 3)
print(x)

tensor([[0.6110, 0.5807, 0.2458],
        [0.8016, 0.2132, 0.6150],
        [0.5390, 0.3417, 0.4696],
        [0.8626, 0.7951, 0.8149],
        [0.8969, 0.3192, 0.2974]])


In [7]:
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [9]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    y = torch.ones_like(x, device=device)
    x = x.to(device)
    z = x + y
    print(z)
    print(z.to('cpu', torch.double))

tensor([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)


In [10]:
x = torch.ones(2, 3, requires_grad = True)
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)


In [11]:
y = x + 2
print(y)

tensor([[3., 3., 3.],
        [3., 3., 3.]], grad_fn=<AddBackward0>)


In [12]:
print(y.grad_fn)

In [14]:
z = y * y * 3 
out = z.mean()
print(z, out)

tensor([[27., 27., 27.],
        [27., 27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [19]:
a = torch.randn(2, 2)
a = ((a * 3)/ (a -1))

print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


In [20]:
out.backward()

In [21]:
print(x.grad)

tensor([[3., 3., 3.],
        [3., 3., 3.]])


In [22]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2
    
print(y)

tensor([882.0911, -10.9804, 864.8436], grad_fn=<MulBackward0>)


In [24]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


In [2]:
import torch
import torch.nn as  nn
import torch.nn.functional as  F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 输入channel：1， 输出channel：6  5X5卷积核
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # y = W * x +b
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        #2 *2 Max pooling
        x = F.max_pool2d(F.relu(self.conv1(x)),(2, 2))
        # 如果是方阵可以一个数字定义
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1, self.num_flat_features(x))
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x )
        
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # 除去处理维度的其他所有维度
        num_features = 1
        
        for s in size:
            num_features *= s
        
        return num_features
    
    
net = Net()
    
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [4]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0537, -0.0567,  0.1269, -0.0219, -0.0546,  0.0304,  0.0335,  0.0443,
         -0.1014,  0.1181]], grad_fn=<AddmmBackward>)


In [5]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [6]:
output = net(input)
target = torch.randn(10)
target = target.view(1, 1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

RuntimeError: shape '[1, 1]' is invalid for input of size 10